# Network and relations making

In [14]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [15]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [16]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "neutral"}
    ],

}

## Other Parameters

In [17]:
INTERVALS = 100
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.05,  # 006
        "prob_tv_spread": 0.05,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.001,  # 001,
        "prob_repost": 0.8,
        "mean_time_connection": 10,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": INTERVALS,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 200,
        "m": 5
    },
    "num_trials": 1
}

# Prob responses parameters

In [18]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )
for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

## Simulation execution

In [19]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', f'{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 10.643027067184448


# Conversation Making

In [20]:
from get_data.get_data import get_pivoted_data, get_type_agents
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from post.transform_time import transform_time
from spelling_checker.spelling_checker import correctness_prompt, correctness_percentage
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.gpt3_5 import send_prompt

## Output sqlite path

In [21]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [22]:
attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)
dict_agents = get_type_agents(analysis_path, sql_table_path)

## Define News

In [23]:
NEWS = 'Dictan prisión preventiva para Pablo Mackenna tras protagonizar accidente de tránsito en estado de ebriedad en Las Condes. '
NEWS_BODY = '''
De acuerdo a los antecedentes que se manejan, Mackenna chocó un taxi ejecutivo en avenida Presidente Errázuriz con Calle Sánchez Fontecilla, provocando graves daños al otro vehículo y dejando una persona lesionada. Al practicarle la alcoholemia, arrojó 1,27 gramos de alcohol por litro de sangre. "Nos tenemos que fijar en la conducta del imputado y en la forma en que pone en riesgo la vida de tercera personas, lo cual efectivamente pasó el día de hoy", aseguró jueza Acevedo. Agregó que "él chocó a un taxista, que el vehículo es su fuente de trabajo, por lo tanto, va a quedar sin poder trabajar, además de la licencia que tiene... También lesionó a otra persona". Asimismo, enfatizó que Mackenna cruzó en luz roja, por lo que "comete una infracción de tránsito, además del manejo en estado de ebriedad". "Se va a acceder a la solicitud de la Fiscalía, y se va a decretar la prisión preventiva", señaló la jueza, precisando que se determinó un plazo de investigación de 90 días.
'''
SHOW_TREE = True

## Create the Network's Tree

In [24]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [25]:
# 24 hours format
TIME_BEG = "10:00"
TIME_END = "23:00"

In [26]:
list_nodes = [root]
num_real_messages = 0
for i in range(len(data)):
    aux = data[i]
    owner = int(aux[0])
    step = int(aux[1])
    step = transform_time(step, TIME_BEG, TIME_END)
    id_message = int(aux[2])
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index] 

    type_agent = dict_agents[owner]
    
    ### Specific for repost attribute
    if attr_dict["repost"] in ["0", "1"]:
        attr_dict["repost"] = bool(int(attr_dict["repost"]))
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")
        
    if not attr_dict["repost"]:
        num_real_messages += 1
    ### 

    message = ''
    news_i = NEWS
    if type_agent == "wise":
        news_i += NEWS_BODY

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, type_agent=type_agent,
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=news_i,
                             **attr_dict
                             )
    )
    

## Associate a Written Post to each Message

In [27]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [30]:
TEMPERATURE = 1.0
list_prompts = []
i = 0
num_llm_errors = 0
MAX_LLM_ERRORS = 10
for node_i in LevelOrderIter(root):
    i += 1
    if node_i == root:
        continue
    elif node_i.repost:
        continue

    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    error_llm = True
    while error_llm:
        try:
            answer = send_prompt(instructions, prompt, temp=TEMPERATURE, max_tokens=1000)
            error_llm = False
        except Exception as err:
            print(f"Error encountered (message {i}/{num_real_messages}):", err)
            num_llm_errors += 1
            error_llm = True
            if num_llm_errors >= MAX_LLM_ERRORS:
                print("Max Errors reached. Local break run")
                break
    if num_llm_errors >= MAX_LLM_ERRORS:
        print("Max Errors reached. Break run")
        break
    correctness = correctness_percentage(answer)
    while correctness >= 0.2:
        print("Correction")
        correction_prompt = correctness_prompt(node_i.news, answer)
        print(correction_prompt)
        answer = send_prompt(instructions, correction_prompt, temp=TEMPERATURE, max_tokens=1000)
        correctness = correctness_percentage(answer)
    
    node_i.set_message(answer)
    list_prompts.append((node_i.name, prompt))

In [31]:
print(i, "/", num_real_messages)

164 / 163


## Plot the Network in a Tree

In [ ]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {node.message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner} (t={node.step})({node.type_agent})({node.parent.name}) {message}")

NEWS: Dictan prisión preventiva para Pablo Mackenna tras protagonizar accidente de tránsito en estado de ebriedad en Las Condes. 
├── 43 (t=13:52)(herd)(0) repost
├── 40 (t=15:37)(wise)(0) repost
│   ├── 79 (t=18:42)(dumb)(2) repost
│   └── 88 (t=20:29)(herd)(2) repost
├── 15 (t=15:57)(wise)(0) repost
│   ├── 50 (t=18:27)(dumb)(3) repost
│   │   ├── 1 (t=19:54)(herd)(8) ''
│   │   │   └── 41 (t=20:05)(dumb)(19) ''
│   │   │       ├── 39 (t=20:42)(dumb)(20) ''
│   │   │       └── 82 (t=22:54)(dumb)(20) repost
│   │   └── 22 (t=22:41)(dumb)(8) repost
│   ├── 2 (t=20:37)(dumb)(3) repost
│   │   ├── 36 (t=21:23)(dumb)(25) ''
│   │   │   └── 27 (t=21:36)(herd)(34) repost
│   │   │       ├── 77 (t=21:48)(herd)(40) repost
│   │   │       └── 73 (t=22:35)(herd)(40) repost
│   │   └── 2 (t=22:11)(dumb)(25) ''
│   └── 11 (t=21:00)(herd)(3) repost
│       ├── 28 (t=21:13)(wise)(30) repost
│       │   └── 12 (t=22:58)(wise)(31) ''
│       ├── 4 (t=21:31)(dumb)(30) repost
│       ├── 9 (t=21:33)(du